# Predicting House Sale Prices

## Introducing the housing data for the city of Ames, Iowa

In [313]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

In [314]:
df = pd.read_csv("AmesHousing.tsv", delimiter= "\t")

In [315]:
def transform_features(df):
    return df

def select_features(df):
    return df[['Gr Liv Area', 'SalePrice']]

def train_and_test(df):
    train = df[:1460]
    test = df[1460:]
    numeric_train = train.select_dtypes(include=['integer', 'float'])
    numeric_test = test.select_dtypes(include=['integer', 'float'])
    
    features = numeric_train.columns.drop("SalePrice")
    lr = LinearRegression()
    lr.fit(train[features], train["SalePrice"])
    predictions = lr.predict(test[features])
    mse = mean_squared_error(test["SalePrice"], predictions)
    rmse = np.sqrt(mse)
    return rmse   

transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df)

rmse

57088.251612639091

## Feature Engineering
* Handle missing values:
  * All columns:
        Drop any with 5% or more missing values for now.
  * Text columns:
        Drop any with 1 or more missing values for now.
  * Numerical columns:
        For columns with missing values, fill in with the most common value in that column


1: All columns: Drop any with 5% or more missing values for now.

In [316]:
# Series object: column name -> number of missing values
num_missing_all = df.isnull().sum()

In [317]:
# Filter Series to columns containing >5% missing values
cols_to_drop = num_missing_all[num_missing_all>(len(df)/20)].index

# Drop those columns from the data frame.
df = df.drop(cols_to_drop, axis=1)

2: Text columns: Drop any with 1 or more missing values for now.

In [318]:
# Series object: column name -> number of missing values
text_cols = df.select_dtypes(include=["object"]).columns
missing_text_count= df[text_cols].isnull().sum()

# Filter Series to columns containing *any* missing values
text_cols_to_drop= missing_text_count[missing_text_count>0].index
df= df.drop(text_cols_to_drop, axis=1)

3: Numerical columns: For columns with missing values, fill in with the most common value in that column

In [319]:
# Compute column-wise missing value counts
num_missing = df.select_dtypes(include=['int', 'float']).isnull().sum()
fixable_numeric_cols = num_missing[(num_missing < len(df)/20) & (num_missing > 0)].sort_values()
fixable_numeric_cols

BsmtFin SF 1       1
BsmtFin SF 2       1
Bsmt Unf SF        1
Total Bsmt SF      1
Garage Cars        1
Garage Area        1
Bsmt Full Bath     2
Bsmt Half Bath     2
Mas Vnr Area      23
dtype: int64

In [320]:
# Compute the most common value for each column in `fixable_nmeric_missing_cols`.
replacement_values_dict = df[fixable_numeric_cols.index].mode().to_dict(orient='records')[0]
replacement_values_dict

{'Bsmt Full Bath': 0.0,
 'Bsmt Half Bath': 0.0,
 'Bsmt Unf SF': 0.0,
 'BsmtFin SF 1': 0.0,
 'BsmtFin SF 2': 0.0,
 'Garage Area': 0.0,
 'Garage Cars': 2.0,
 'Mas Vnr Area': 0.0,
 'Total Bsmt SF': 0.0}

In [321]:
# Replace missing values with the most common values for each column.
df = df.fillna(replacement_values_dict)

In [322]:
# Verify that every column has 0 missing values
df.isnull().sum().value_counts()

0    64
dtype: int64

Create new features that better capture the information in some of the features.

In [323]:
years_sold = df['Yr Sold'] - df['Year Built']
years_sold[years_sold < 0]

2180   -1
dtype: int64

In [324]:
years_since_remod = df['Yr Sold'] - df['Year Remod/Add']
years_since_remod[years_since_remod < 0]

1702   -1
2180   -2
2181   -1
dtype: int64

In [325]:
# Create new columns for "Years Before Sale" and "Years since Remod"
df['Years Before Sale'] = years_sold
df['Years Since Remod'] = years_since_remod

In [326]:
# Drop rows with negative values for both these features. 
df= df.drop([1702,2180,2181], axis=0)

# No longer need original year columns
df = df.drop(["Year Built", "Year Remod/Add"], axis = 1)

Drop columns that:

    * that aren't useful for ML
    * leak data about the final sale

In [327]:
# Drop columns that aren't useful for ML
df = df.drop(["PID", "Order"], axis=1)

# Drop columns that leak info about the final sale
df = df.drop(["Mo Sold", "Sale Condition", "Sale Type", "Yr Sold"], axis=1)

## Updating transform_features() function

In [328]:
def transform_features(df):
    num_missing_all = df.isnull().sum()
    cols_to_drop = num_missing_all[num_missing_all>(len(df)/20)].index
    df = df.drop(cols_to_drop, axis=1)
    
    text_cols = df.select_dtypes(include=["object"]).columns
    missing_text_count= df[text_cols].isnull().sum()
    text_cols_to_drop= missing_text_count[missing_text_count>0].index
    df= df.drop(text_cols_to_drop, axis=1)
    
    num_missing = df.select_dtypes(include=['int', 'float']).isnull().sum()
    fixable_numeric_cols = num_missing[(num_missing < len(df)/20) & (num_missing > 0)].sort_values()
        
    replacement_values_dict = df[fixable_numeric_cols.index].mode().to_dict(orient='records')[0]
    df = df.fillna(replacement_values_dict)
    
    df['Years Before Sale'] = df['Yr Sold'] - df['Year Built']
    df['Years Since Remod'] = df['Yr Sold'] - df['Year Remod/Add']
    
    df= df.drop([1702,2180,2181], axis=0)
    df = df.drop(["PID", "Order", "Mo Sold", "Sale Condition", "Sale Type", "Year Built", "Year Remod/Add"], axis = 1)
    return df

def select_features(df):
    return df[['Gr Liv Area', 'SalePrice']]

def train_and_test(df):
    train = df[:1460]
    test = df[1460:]
    numeric_train = train.select_dtypes(include=['integer', 'float'])
    numeric_test = test.select_dtypes(include=['integer', 'float'])
    
    features = numeric_train.columns.drop("SalePrice")
    lr = LinearRegression()
    lr.fit(train[features], train["SalePrice"])
    predictions = lr.predict(test[features])
    mse = mean_squared_error(test["SalePrice"], predictions)
    rmse = np.sqrt(mse)
    return rmse   

df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df)

rmse

55275.367312413073

## Feature Selection

In [329]:
numerical_df = df.select_dtypes(include=['integer', 'float'])
corr_mat= numerical_df.corr()["SalePrice"].sort_values()
corr_mat

PID               -0.246521
Enclosed Porch    -0.128787
Kitchen AbvGr     -0.119814
Overall Cond      -0.101697
MS SubClass       -0.085092
Low Qual Fin SF   -0.037660
Bsmt Half Bath    -0.035835
Order             -0.031408
Yr Sold           -0.030569
Misc Val          -0.015691
BsmtFin SF 2       0.005891
3Ssn Porch         0.032225
Mo Sold            0.035259
Pool Area          0.068403
Screen Porch       0.112151
Bedroom AbvGr      0.143913
Bsmt Unf SF        0.182855
Lot Area           0.266549
2nd Flr SF         0.269373
Bsmt Full Bath     0.276050
Half Bath          0.285056
Open Porch SF      0.312951
Wood Deck SF       0.327143
Lot Frontage       0.357318
BsmtFin SF 1       0.432914
Fireplaces         0.474558
TotRms AbvGrd      0.495474
Mas Vnr Area       0.508285
Garage Yr Blt      0.526965
Year Remod/Add     0.532974
Full Bath          0.545604
Year Built         0.558426
1st Flr SF         0.621676
Total Bsmt SF      0.632280
Garage Area        0.640401
Garage Cars        0

In [330]:
abs_corr_coeffs= numerical_df.corr()["SalePrice"].abs().sort_values()
abs_corr_coeffs

BsmtFin SF 2       0.005891
Misc Val           0.015691
Yr Sold            0.030569
Order              0.031408
3Ssn Porch         0.032225
Mo Sold            0.035259
Bsmt Half Bath     0.035835
Low Qual Fin SF    0.037660
Pool Area          0.068403
MS SubClass        0.085092
Overall Cond       0.101697
Screen Porch       0.112151
Kitchen AbvGr      0.119814
Enclosed Porch     0.128787
Bedroom AbvGr      0.143913
Bsmt Unf SF        0.182855
PID                0.246521
Lot Area           0.266549
2nd Flr SF         0.269373
Bsmt Full Bath     0.276050
Half Bath          0.285056
Open Porch SF      0.312951
Wood Deck SF       0.327143
Lot Frontage       0.357318
BsmtFin SF 1       0.432914
Fireplaces         0.474558
TotRms AbvGrd      0.495474
Mas Vnr Area       0.508285
Garage Yr Blt      0.526965
Year Remod/Add     0.532974
Full Bath          0.545604
Year Built         0.558426
1st Flr SF         0.621676
Total Bsmt SF      0.632280
Garage Area        0.640401
Garage Cars        0

In [331]:
abs_corr_coeffs[abs_corr_coeffs<0.4]

BsmtFin SF 2       0.005891
Misc Val           0.015691
Yr Sold            0.030569
Order              0.031408
3Ssn Porch         0.032225
Mo Sold            0.035259
Bsmt Half Bath     0.035835
Low Qual Fin SF    0.037660
Pool Area          0.068403
MS SubClass        0.085092
Overall Cond       0.101697
Screen Porch       0.112151
Kitchen AbvGr      0.119814
Enclosed Porch     0.128787
Bedroom AbvGr      0.143913
Bsmt Unf SF        0.182855
PID                0.246521
Lot Area           0.266549
2nd Flr SF         0.269373
Bsmt Full Bath     0.276050
Half Bath          0.285056
Open Porch SF      0.312951
Wood Deck SF       0.327143
Lot Frontage       0.357318
Name: SalePrice, dtype: float64

In [332]:
# Drop columns with less than 0.4 correlation with SalePrice.
columns= abs_corr_coeffs[abs_corr_coeffs < 0.4].index
for col in columns:
    if col in transform_df.columns:
        transform_df= transform_df.drop(col, axis=1)

Which categorical columns should we keep?

In [333]:
# List of column names that are categorical.
nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]

* Which columns are currently numerical but need to be encoded as categorical instead (because the numbers don't have any semantic meaning)?
* If a categorical column has hundreds of unique values (or categories), should we keep it? When we dummy code this column, hundreds of columns will need to be added back to the data frame.

In [336]:
# Which categorical columns have we still carried with us? 
transform_cat_cols = []
for col in nominal_features:
    if col in transform_df.columns:
        transform_cat_cols.append(col)

transform_cat_cols        

['MS Zoning',
 'Street',
 'Land Contour',
 'Lot Config',
 'Neighborhood',
 'Condition 1',
 'Condition 2',
 'Bldg Type',
 'House Style',
 'Roof Style',
 'Roof Matl',
 'Exterior 1st',
 'Exterior 2nd',
 'Foundation',
 'Heating',
 'Central Air']

In [337]:
# How many unique values in each categorical column?
uniqueness_counts = transform_df[transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()
uniqueness_counts

Street           2
Central Air      2
Land Contour     4
Lot Config       5
Bldg Type        5
Roof Style       6
Foundation       6
Heating          6
MS Zoning        7
Condition 2      8
House Style      8
Roof Matl        8
Condition 1      9
Exterior 1st    16
Exterior 2nd    17
Neighborhood    28
dtype: int64

In [338]:
# Aribtrary cutoff of 10 unique values.
drop_nonuniq_cols = uniqueness_counts[uniqueness_counts > 10].index
transform_df = transform_df.drop(drop_nonuniq_cols, axis=1)

In [339]:
# Select just the remaining text columns and convert to categorical.
text_cols = transform_df.select_dtypes(include=['object']).columns
for col in text_cols:
    transform_df[col] = transform_df[col].astype('category')
    
# Create dummy columns and add back to the dataframe!
transform_df = pd.concat([
    transform_df, 
    pd.get_dummies(transform_df.select_dtypes(include=['category']))
], axis=1)

## Updating select_features function.

In [340]:
def transform_features(df):
    num_missing_all = df.isnull().sum()
    cols_to_drop = num_missing_all[num_missing_all>(len(df)/20)].index
    df = df.drop(cols_to_drop, axis=1)
    
    text_cols = df.select_dtypes(include=["object"]).columns
    missing_text_count= df[text_cols].isnull().sum()
    text_cols_to_drop= missing_text_count[missing_text_count>0].index
    df= df.drop(text_cols_to_drop, axis=1)
    
    num_missing = df.select_dtypes(include=['int', 'float']).isnull().sum()
    fixable_numeric_cols = num_missing[(num_missing < len(df)/20) & (num_missing > 0)].sort_values()
        
    replacement_values_dict = df[fixable_numeric_cols.index].mode().to_dict(orient='records')[0]
    df = df.fillna(replacement_values_dict)
    
    df['Years Before Sale'] = df['Yr Sold'] - df['Year Built']
    df['Years Since Remod'] = df['Yr Sold'] - df['Year Remod/Add']
    
    df= df.drop([1702,2180,2181], axis=0)
    df = df.drop(["PID", "Order", "Mo Sold", "Sale Condition", "Sale Type", "Year Built", "Year Remod/Add"], axis = 1)
    return df

def select_features(df, coeff_threshold=0.4, uniq_threshold=10):
    numerical_df = df.select_dtypes(include=['int', 'float'])
    abs_corr_coeffs = numerical_df.corr()['SalePrice'].abs().sort_values()
    df = df.drop(abs_corr_coeffs[abs_corr_coeffs < coeff_threshold].index, axis=1)
    
    nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]
    
    transform_cat_cols = []
    for col in nominal_features:
        if col in df.columns:
            transform_cat_cols.append(col)

    uniqueness_counts = df[transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()
    drop_nonuniq_cols = uniqueness_counts[uniqueness_counts > 10].index
    df = df.drop(drop_nonuniq_cols, axis=1)
    
    text_cols = df.select_dtypes(include=['object'])
    for col in text_cols:
        df[col] = df[col].astype('category')
    df = pd.concat([df, pd.get_dummies(df.select_dtypes(include=['category']))], axis=1)
    
    return df

def train_and_test(df, k=0):
    numeric_df = df.select_dtypes(include=['integer', 'float'])
    features = numeric_df.columns.drop("SalePrice")
    lr = LinearRegression()
    
    # When k equals 0, holdout validation .
    if k==0:
        train = df[:1460]
        test = df[1460:]  
        lr.fit(train[features], train["SalePrice"])
        predictions = lr.predict(test[features])
        mse = mean_squared_error(test["SalePrice"], predictions)
        rmse = np.sqrt(mse)
        return rmse   
    
    # When k equals 1, perform simple cross validation
    if k==1: 
        shuffled_df = df.iloc[np.random.permutation(len(numeric_df))]
        fold_one = shuffled_df[0:1460]
        fold_two= shuffled_df[1460:]
        rmses=[]
        
        lr.fit(fold_one[features], fold_one["SalePrice"])
        predictions = lr.predict(fold_two[features])
        mse_one = mean_squared_error(fold_two["SalePrice"], predictions)
        rmse_one = np.sqrt(mse_one)
        rmses.append(rmse_one)
        
        lr.fit(fold_two[features], fold_two["SalePrice"])
        predictions = lr.predict(fold_one[features])
        mse_two = mean_squared_error(fold_one["SalePrice"], predictions)
        rmse_two = np.sqrt(mse_one)
        rmses.append(rmse_two)
        
        avg_rmse = np.mean(rmses)
        return avg_rmse
    
    # When K>1 implement k-fold cross validation using k folds
    else: 
        kf= KFold(n_splits=k, shuffle=True)
        rmse_values=[]
        for train_index, test_index in kf.split(df):
            train= df.iloc[train_index]
            test = df.iloc[test_index]
            lr.fit(train[features], train["SalePrice"])
            predictions = lr.predict(test[features])
            mse = mean_squared_error(test["SalePrice"], predictions)
            rmse = np.sqrt(mse)
            rmse_values.append(rmse)
        print(rmse_values)
        avg_rmse= np.mean(rmse_values)
        return avg_rmse       

In [341]:
df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df, k=4)

rmse

[27824.903234131503, 36612.017868973628, 25767.771045441234, 26462.813070505119]


29166.876304762871

In [342]:
df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df, k=0)

rmse

33367.287183402834

In [343]:
df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df, k=1)

rmse

26075.079681164883